In [ ]:
!nvidia-smi

Sat Nov  7 06:25:06 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
#!pip install transforms3d
!pip install keras-tcn

     |████████████████████████████████| 378kB 5.2MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal, spatial, stats
from fastdtw import fastdtw
from scipy.signal import find_peaks
import pywt
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
import keras
from scipy.interpolate import CubicSpline      # for warping
#from transforms3d.axangles import axangle2mat  # for rotation
from keras.layers import Input, Dense, Conv1D, Conv2D, MaxPooling2D, MaxPooling1D, Flatten, Dropout, BatchNormalization, Bidirectional, LSTM
from keras.models import Model
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix
from keras.optimizers import Adam
from keras.losses import CategoricalCrossentropy, BinaryCrossentropy
from keras.metrics import Accuracy, Precision, Recall
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tcn import TCN, tcn_full_summary
from keras import Sequential

In [ ]:
os.chdir('drive/My Drive/Weave/Earables Project/Final Experiments/')

In [ ]:
data = pickle.load(open('data/processed_df.pkl', 'rb'))
fold_idx = pickle.load(open('data/5_fold_idx.pkl', 'rb'))

In [ ]:
print('subjects:', np.unique(data.subject))
print('sessions:', np.unique(data.session))

subjects: ['11' '12' '13' '15' '17' '2' '4' '6' '7']
sessions: ['1' '2' '3' '4' '5' '6' '7']


In [ ]:
test_subject = '6'
test_session = '5'

train_data = data[data.subject != test_subject]
test_data = data[data.subject == test_subject]

X_train = train_data.iloc[:,:-3].to_numpy()
y_train = train_data.iloc[:,-3:-2]
X_test = test_data.iloc[:,:-3].to_numpy()
y_test = test_data.iloc[:,-3:-2]
ohe = OneHotEncoder()
y_train = ohe.fit_transform(y_train).toarray()
y_test = ohe.transform(y_test).toarray()

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(82848, 12) (82848, 32)
(10800, 12) (10800, 32)


In [ ]:
n_classes = y_train.shape[1]
print('num. of classes: {}'.format(n_classes))

num. of classes: 32


In [ ]:
class Data_Generator(keras.utils.Sequence):
    
    def __init__(self, data_x, labels, batch_size, augment=False) :
        self.data_x = data_x
        self.labels = labels
        self.batch_size = batch_size
        self.augment = augment
    
    def __len__(self):
        return (np.ceil(len(self.data_x) / float(self.batch_size))).astype(np.int)
  
    def __getitem__(self, idx) :
        x = self.data_x[idx * self.batch_size : (idx+1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
        
        batch_x = np.ones((x.shape[0], 100, x.shape[1]))

        for i in range(len(x)):
            if self.augment:
                aug = np.random.choice(range(4))
            else:
                aug = 3
            #print(aug)
            temp = np.array([np.array(a) for a in x[i]])
            print(temp.shape)
            if aug == 0:
                batch_x[i] = np.hstack([self.DA_TimeWarp(temp[3*j:3*(j+1)].T) for j in range(len(temp)//3)])
            elif aug == 1:
                batch_x[i] = np.hstack([self.DA_MagWarp(temp[3*j:3*(j+1)].T, 0.2) for j in range(len(temp)//3)])
            elif aug == 2:
                batch_x[i] = np.hstack([self.DA_Rotation(temp[3*j:3*(j+1)].T) for j in range(len(temp)//3)])
            else:
                #min_max_scaler = preprocessing.MinMaxScaler()
                for j in range(len(temp)):
                    print(temp[j].shape)
                batch_x[i] = np.array([(j - j.min())/(j.max() - j.min()) for j in temp]).T

        return np.asarray(batch_x), np.asarray(batch_y)

    def DistortTimesteps(self, X, sigma=0.2):
        tt = self.GenerateRandomCurves(X, sigma) # Regard these samples aroun 1 as time intervals
        tt_cum = np.cumsum(tt, axis=0)        # Add intervals to make a cumulative graph
        # Make the last value to have X.shape[0]
        t_scale = [(X.shape[0]-1)/tt_cum[-1,0],(X.shape[0]-1)/tt_cum[-1,1],(X.shape[0]-1)/tt_cum[-1,2]]
        tt_cum[:,0] = tt_cum[:,0]*t_scale[0]
        tt_cum[:,1] = tt_cum[:,1]*t_scale[1]
        tt_cum[:,2] = tt_cum[:,2]*t_scale[2]
        return tt_cum

    def DA_TimeWarp(self, X, sigma=0.2):
        tt_new = self.DistortTimesteps(X, sigma)
        X_new = np.zeros(X.shape)
        x_range = np.arange(X.shape[0])
        X_new[:,0] = np.interp(x_range, tt_new[:,0], X[:,0])
        X_new[:,1] = np.interp(x_range, tt_new[:,1], X[:,1])
        X_new[:,2] = np.interp(x_range, tt_new[:,2], X[:,2])
        return X_new

    def DA_Rotation(self, X):
        axis = np.random.uniform(low=-1, high=1, size=X.shape[1])
        angle = np.random.uniform(low=-np.pi, high=np.pi)
        return np.matmul(X , axangle2mat(axis,angle))

    def GenerateRandomCurves(self, X, sigma=0.2, knot=4):
        xx = (np.ones((X.shape[1],1))*(np.arange(0,X.shape[0], (X.shape[0]-1)/(knot+1)))).transpose()
        yy = np.random.normal(loc=1.0, scale=sigma, size=(knot+2, X.shape[1]))
        x_range = np.arange(X.shape[0])
        cs_x = CubicSpline(xx[:,0], yy[:,0])
        cs_y = CubicSpline(xx[:,1], yy[:,1])
        cs_z = CubicSpline(xx[:,2], yy[:,2])
        return np.array([cs_x(x_range),cs_y(x_range),cs_z(x_range)]).transpose()
    
    def DA_MagWarp(self, X, sigma):
        return X * self.GenerateRandomCurves(X, sigma)

In [ ]:
import tensorflow
model = tensorflow.keras.Sequential(name = 'conv2d_12ch_bilstm')
model.add(tensorflow.keras.Input(shape=(100, 12)))
model.add(tensorflow.keras.layers.Bidirectional(tensorflow.keras.layers.LSTM(units=64, return_sequences=True)))
model.add(tensorflow.keras.layers.Bidirectional(tensorflow.keras.layers.LSTM(units=128)))
model.add(tensorflow.keras.layers.Dropout(rate=0.5))
model.add(tensorflow.keras.layers.Dense(units=512, activation='relu'))
model.add(tensorflow.keras.layers.Dense(n_classes, activation='sigmoid'))

In [6]:
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

batch_size = 32
input_shape = (100, 12)
img_input = Input(shape=(100, 12))

x = TCN(return_sequences=False)(img_input)
# x = TCN(return_sequences=False)(x)
x = Dense(32, activation='sigmoid', name='output')(x)

model = Model(img_input, x, name='tcn_12ch')
model.summary()

Model: "tcn_12ch"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 12)]         0         
_________________________________________________________________
tcn (TCN)                    (None, 64)                93248     
_________________________________________________________________
output (Dense)               (None, 32)                2080      
Total params: 95,328
Trainable params: 95,328
Non-trainable params: 0
_________________________________________________________________


In [7]:
tcn_full_summary(model, expand_residual_blocks=True)

Model: "tcn_12ch"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 12)]         0         
_________________________________________________________________
matching_conv1D (Conv1D)     (None, 100, 64)           832       
_________________________________________________________________
activation_2 (Activation)    (None, 100, 64)           0         
_________________________________________________________________
conv1D_0 (Conv1D)            (None, 100, 64)           1600      
_________________________________________________________________
activation (Activation)      (None, 100, 64)           0         
_________________________________________________________________
spatial_dropout1d (SpatialDr (None, 100, 64)           0         
_________________________________________________________________
conv1D_1 (Conv1D)            (None, 100, 64)           825

In [ ]:
batch_size = 32
train_gen = Data_Generator(X_train, y_train, batch_size)
test_gen = Data_Generator(X_test, y_test, batch_size)

In [ ]:
model_name = 'tcn_sigmoid_normalized_sub_'+test_subject
optimizer = Adam(learning_rate=0.00001)
# loss = BinaryCrossentropy()
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics='accuracy')
checkpoint = ModelCheckpoint('checkpoint/across_user_accuracy/{}.h5'.format(model_name), monitor='val_loss', verbose=1, mode='min',save_best_only=True)
earlystopping = EarlyStopping(monitor='val_loss', patience=20, verbose=1)
reducelr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, min_lr=0.000001)

In [ ]:
tmp = []
for i in range(len(X_train)):
    ch = []
    for j in range(12):
        ch.append((X_train[i][j] - X_train[i][j].min())/(X_train[i][j].max() - X_train[i][j].min()))
    ch = np.array(ch).T
    tmp.append(ch)
X_train = np.array(tmp)

tmp = []
for i in range(len(X_test)):
    ch = []
    for j in range(12):
        ch.append((X_test[i][j] - X_test[i][j].min())/(X_test[i][j].max() - X_test[i][j].min()))
    ch = np.array(ch).T
    tmp.append(ch)
X_test = np.array(tmp)
del tmp, ch
print(X_train.shape)
print(X_test.shape)

(82848, 100, 12)
(10800, 100, 12)


In [ ]:
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    batch_size=batch_size,
                    epochs=100,
                    # workers=4,
                    callbacks=[checkpoint, earlystopping, reducelr],
                    shuffle=True)

In [ ]:
from xgboost import XGBClassifier
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
data = pickle.load(open('data/all_features_df.pkl', 'rb'))

In [ ]:
labels = data.iloc[:,-3:-2]
ohe = OneHotEncoder()
ohe.fit(labels)

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=True)

In [ ]:
subjects = list(np.unique(data.subject))
# print(subjects)
test_i = 7
test_sub = subjects[test_i]
test_session = '7'

train_data = data[data.session != test_session]
test_data = data[data.session == test_session]

X_train = train_data.iloc[:,:-3]
y_train = train_data.iloc[:,-3:-2]
X_test = test_data.iloc[:,:-3]
y_test = test_data.iloc[:,-3:-2]
ohe = OneHotEncoder()
y_train = ohe.fit_transform(y_train).toarray()
y_test = ohe.transform(y_test).toarray()

In [ ]:
print(X_test.shape)

(13200, 852)


In [ ]:
imp_features = pickle.load(open('data/important_features.pkl', 'rb'))
X_train = X_train[imp_features]
X_test = X_test[imp_features]

In [ ]:
X_train = np.nan_to_num(X_train.to_numpy(dtype = np.float64))
X_test = np.nan_to_num(X_test.to_numpy(dtype = np.float64))

In [ ]:
xgb_classifier = OneVsRestClassifier(XGBClassifier())
xgb_classifier.fit(X_train, y_train)

OneVsRestClassifier(estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                            colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=1, gamma=0,
                                            learning_rate=0.1, max_delta_step=0,
                                            max_depth=3, min_child_weight=1,
                                            missing=None, n_estimators=100,
                                            n_jobs=1, nthread=None,
                                            objective='binary:logistic',
                                            random_state=0, reg_alpha=0,
                                            reg_lambda=1, scale_pos_weight=1,
                                            seed=None, silent=None, subsample=1,
                                            verbosity=1),
                    n_jobs=None)

In [ ]:
y_pred_imp_xg = xgb_classifier.predict(X_test)
print(classification_report(y_test, y_pred_imp_xg))